# pyAFQ tractography pipeline (adapted for Moramay Ramos-Flores)

This notebook explains **step by step** what each import, function, and parameter does.
All explanations are included directly above each code cell.
You must have:
- pyAFQ installed (`pip install pyafq`)
- Your diffusion data organized in **BIDS format**


## 1. Import libraries

Here we import all the modules needed. Each one has a specific role.

- `AFQ.api.bundle_dict`: lets you load default bundles or create custom ones.
- `AFQ.data.fetch`: contains the datasets (e.g., `stanford_hardi`).
- `GroupAFQ`: main class to run pyAFQ on a whole BIDS dataset.
- `ImageFile` and `RoiImage`: used to define masks, ROIs, and their logic (include/exclude).
- `os` and `os.path`: used to build file paths safely.


In [ ]:
import AFQ.api.bundle_dict as abd
import AFQ.data.fetch as afd
from AFQ.api.group import GroupAFQ
import AFQ.utils.streamlines as aus
from AFQ.definitions.image import ImageFile, RoiImage
import AFQ.definitions.image as afm
import os
import os.path as op

## 2. Select the bundles to analyze

Here we select **a subset** of the 18 default bundles provided by pyAFQ.

The list `other_bundles` contains the names exactly as pyAFQ defines them. These names must match the keys in the `default18_bd()` dictionary.

`abd.default18_bd()` → returns *all* 18 bundles defined by pyAFQ.

`abd.default18_bd()[other_bundles]` → extracts only the bundles we want.


In [ ]:
other_bundles = [
    "Left Arcuate", "Right Arcuate",
    "Left Inferior Fronto-occipital", "Right Inferior Fronto-occipital",
    "Left Inferior Longitudinal", "Right Inferior Longitudinal",
    "Left Superior Longitudinal", "Right Superior Longitudinal",
    "Left Uncinate", "Right Uncinate"
]

bundles = abd.default18_bd()[other_bundles]
bundles.bundle_names

## 3. Define brain mask logic

pyAFQ needs a **brain mask** for tractography.

- `suffix='mask'` tells pyAFQ to search for files ending in `*mask.nii.gz`
- `filters={'scope': 'freesurfer'}` restricts the search to masks generated by FreeSurfer.
- `exclusive_labels=[0]` tells pyAFQ to treat label 0 (background) as excluded.


In [ ]:
brain_mask_definition = afm.LabelledImageFile(
    suffix='mask',
    filters={'scope': 'freesurfer'},
    exclusive_labels=[0]
)

## 4. Initialize GroupAFQ

This is the **core object** that runs the entire bundle extraction pipeline.

### Explanation of parameters:
- `op.join(afd.afq_home,'stanford_hardi')` → loads the Stanford HARDI the dataset.
- `bundle_info=bundles` → the subset of bundles we selected.
- `preproc_pipeline='vistasoft'` → tells pyAFQ how the dataset the DWI images was preprocessed.
- `scalars=["dki_fa"]` → pyAFQ extracts FA along the tract.
- `tracking_params` → controls tractography:
  - `n_seeds=1000000`: how many seeds per subject.
  - `random_seeds=True`: seeds are randomly distributed inside the seed mask.
  - `seed_mask=RoiImage(...)`: defines where streamlines can begin.

pyAFQ automatically identifies all DWI and anatomical files following the BIDS specification. No manual file selection is required.

Because the analysis uses dki_fa, pyAFQ automatically fits a Diffusion Kurtosis Imaging (DKI) model using weighted least-squares (default DIPY implementation). Additional DKI-derived metrics (mean, axial, and radial kurtosis) are computed implicitly.
Registration follows pyAFQ’s default two-step pipeline: affine alignment of the subject’s DKI-FA map to the template, followed by SyN non-linear registration.

Fiber orientation reconstruction is performed using pyAFQ’s default single-tissue Constrained Spherical Deconvolution model (CSD; response function estimated using the Tournier method, SH order l=8).

Streamlines were generated using probabilistic CSD tracking with pyAFQ’s default settings (0.5 mm step size, 30° curvature threshold, 30–250 mm allowed lengths).
1,000,000 random seeds were used, and waypoint/endpoints masks were enforced.

Bundle segmentation follows the default pyAFQ logic, combining waypoint-based inclusion/exclusion, probabilistic atlas matching, and automatic removal of anatomically inconsistent streamlines.

Tract cleaning applies the standard pyAFQ procedure: length-based trimming (5%), Mahalanobis distance thresholding (5), and resampling of streamlines to 100 equidistant nodes.



In [ ]:
myafq = GroupAFQ(
    op.join(afd.afq_home, 'stanford_hardi'),
    bundle_info=bundles,
    brain_mask_definition=brain_mask_definition,
    preproc_pipeline='vistasoft',
    scalars=["dki_fa"],
    tracking_params={
        "n_seeds": 1000000,
        "random_seeds": True,
        "seed_mask": RoiImage(use_waypoints=True, use_endpoints=True)
    }
)

## 5. Export results

`export_all()` generates outputs including:
- Tract profiles (.csv with tractID, nodeID, dki_fa, subjetID,sessionID)
- Streamlines
- Bundle segmentations

Here we disable visualizations to speed up processing.


In [ ]:
myafq.export_all(
    viz=False,
    afqbrowser=False,
    xforms=False,
    indiv=False
)